# Llama 2 on Haystack!
This notebook contains my hacky experiments in which I try to load and use [Llama2](https://ai.meta.com/llama/) with [Haystack](https://github.com/deepset-ai/haystack), the NLP/LLM framework.


*It's nothing official or well refined, but perhaps it may be useful to other people experimenting.*

![](https://media.istockphoto.com/id/1335757394/video/alpaca-animals-close-up-of-haystack-and-chewing-action.jpg?s=640x640&k=20&c=Ar3vSZooIJ1Izo8MDXuLOtf27AiIvSwIx4R0kMGGmLo=)

## Install Transformers and related dependencies

In [1]:
# we need to install transformers from the main branch to correctly handle Tensor Parallelism (https://github.com/huggingface/transformers/pull/24906)
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes accelerate

DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: tensorflow-gpu 2.11.0 has a non-standard dependency specifier platform_system!="Darwin". pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow-gpu or contact the author to suggest that they release a version with a con

## Load Llama2 using Transformers

### Llama 2 access
- You need to request access to Llama 2 at https://ai.meta.com/resources/models-and-libraries/llama-downloads/
- Then also request access to the Hugging Face models (with the same e-mail address): https://huggingface.co/meta-llama
- You will receive some acceptance e-mail messages!


In [2]:
# A valid (free) Hugging Face Access Token (https://huggingface.co/settings/tokens)
import os 
hf_token=os.environ["HF_TOKEN"]

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Llama-2-13b-chat-hf"

# load the model using 4bit quantization (https://huggingface.co/blog/4bit-transformers-bitsandbytes)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, use_auth_token=hf_token)
# disable Tensor Parallelism (https://github.com/huggingface/transformers/pull/24906)
model.config.pretraining_tp=1

tokenizer = AutoTokenizer.from_pretrained (model_id, use_auth_token=hf_token)

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
# quick sanity check
input_text = "Describe the solar system."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(input_ids, max_length=50)
print(tokenizer.decode(outputs[0]))

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


<s> Describe the solar system.

The solar system consists of eight planets, dwarf planets, asteroids, comets, and other celestial bodies that orbit the Sun. The Sun is the center of the solar system


In [10]:
def get_response(input_text,max_length=50):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, max_length=max_length)
    return print(tokenizer.decode(outputs[0]))
    

In [16]:
get_response("Who is the President of US?")

<s> Who is the President of US?
The current President of the United States is Joe Biden. He was inaugurated as the 46th President of the United States on January 20, 2021, and is


In [14]:
get_response("Explain material weakness in accounting in detail, preferably with an example", 1024)

<s> Explain material weakness in accounting in detail, preferably with an example.

Material weakness in accounting refers to a deficiency or shortcoming in the internal control over financial reporting that can lead to material misstatements in the financial statements. A material weakness is considered to be a significant deficiency or lack of internal control that could result in materially misstating the financial statements.

For example, let's say that a company has a material weakness in its accounting for inventory. The company's internal control over inventory is not effective in preventing or detecting material misstatements in the financial statements. As a result, the company's financial statements may contain material misstatements related to inventory balances, cost of goods sold, or other financial statement line items.

To illustrate this further, let's say that the company has a large number of inventory items that are not properly recorded or accounted for. This could

In [18]:
%%time
get_response("When should I explore the white rim road in Canyon Lands?", 512)

<s> When should I explore the white rim road in Canyon Lands?

The White Rim Road is a 100-mile dirt road that runs along the rim of Canyonlands National Park in Utah. The road is open to vehicles year-round, but the best time to explore it depends on your preferences and the conditions. Here are some factors to consider:

1. Weather: The weather in Canyonlands can be extreme, with hot summers and cold winters. The best time to explore the White Rim Road is during the cooler months (September to May), when the temperatures are more moderate.
2. Crowds: The White Rim Road is a popular destination, and it can get crowded during peak season (March to May and September to November). If you prefer fewer crowds, consider visiting during the shoulder season (June to August and December to February).
3. Road conditions: The White Rim Road is a dirt road, and it can be rough and dusty. The road is usually in good condition, but it can be slippery during wet weather. If you're planning to drive 

## Install Haystack (minimal version)

To make sure that the Haystack installation does not interfere with this particular Transformers installation, we manually prepare a minimal list of dependencies (https://github.com/deepset-ai/haystack/pull/5101).

In [11]:
# for unknown reasons, we should reset the locale
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [12]:
with open('haystack-minimal-requirements.txt','w') as fo:
  fo.write("""tokenizers
pydantic
pandas
rank_bm25
lazy-imports==0.3.1
prompthub-py==4.0.0
platformdirs
tqdm
networkx
quantulum3
posthog
huggingface-hub>=0.5.0
tenacity
sseclient-py
more_itertools
boilerpy3
tiktoken>=0.3.2
jsonschema
events
requests-cache<1.0.0
pillow
click""")

!pip install -q --no-deps git+https://github.com/deepset-ai/haystack.git && pip install -qr haystack-minimal-requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Initialize and try the PromptNode
The PromptNode is an easy-to-use and customizable component powered by LLMs (like Llama2).

It can run on its own or in your Pipelines for various NLP tasks.

[PromptNode docs](https://docs.haystack.deepset.ai/docs/prompt_node)

In [13]:
from haystack.nodes import PromptNode,PromptModel

# exotic configuration based on model_kwargs
# inspiration: https://docs.haystack.deepset.ai/docs/prompt_node#using-models-not-supported-in-hugging-face-transformers
pn = PromptNode("meta-llama/Llama-2-13b-chat-hf",
                max_length=1000,
                model_kwargs={'model':model,
                              'tokenizer':tokenizer,
                              'task_name':'text2text-generation',
                              'device':None, # placeholder needed to make the underlying HF Pipeline work
                              'stream':True})


The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [14]:
# simply call the PromptNode

pn("What's the coolest city in Italy? Explain reasons why")

<s> What's the coolest city in Italy? Explain reasons why you think so.
What's the coolest city in Italy? Explain reasons why you think so.
Italy is a country known for its rich history, art, architecture, fashion, and cuisine. Each city in Italy has its own unique charm and character, making it difficult to pinpoint the coolest one. However, based on various factors such as cultural offerings, nightlife, food scene, and overall vibe, here are some reasons why I think Florence is the coolest city in Italy:

1. Art and History: Florence is home to some of the world's most famous museums, galleries, and landmarks, including the Uffizi Gallery, the Accademia Gallery (where Michelangelo's David is housed), and the Duomo. The city is steeped in history and art, making it a paradise for culture lovers.
2. Fashion: Florence is known for its high-end fashion scene, with iconic brands like Gucci, Prada, and Salvatore Ferragamo originating from the city. The city is also home to numerous boutiqu

["What's the coolest city in Italy? Explain reasons why you think so.\nWhat's the coolest city in Italy? Explain reasons why you think so.\nItaly is a country known for its rich history, art, architecture, fashion, and cuisine. Each city in Italy has its own unique charm and character, making it difficult to pinpoint the coolest one. However, based on various factors such as cultural offerings, nightlife, food scene, and overall vibe, here are some reasons why I think Florence is the coolest city in Italy:\n\n1. Art and History: Florence is home to some of the world's most famous museums, galleries, and landmarks, including the Uffizi Gallery, the Accademia Gallery (where Michelangelo's David is housed), and the Duomo. The city is steeped in history and art, making it a paradise for culture lovers.\n2. Fashion: Florence is known for its high-end fashion scene, with iconic brands like Gucci, Prada, and Salvatore Ferragamo originating from the city. The city is also home to numerous bout

## Conversational Agent

We simply build a Chat App with Memory.

You can find information and a more complex application explained [in this Haystack Tutorial](https://haystack.deepset.ai/tutorials/24_building_chat_app).

In [15]:
from haystack.agents.conversational import ConversationalAgent

# We need to design a specific prompt template, suitable for Llama2
# inspiration: https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat/tree/main
prompt_template="""
[INST] <<SYS>>
You are a helpful assistant who writes short answers.
<</SYS>>\n\n
{memory}</s><s> [INST] {query} [/INST]
"""

conversational_agent = ConversationalAgent(
    prompt_node=pn,
    prompt_template=prompt_template,
)

In [17]:
while True:
  query=input("\nHuman (type 'exit' or 'quit' to quit): ")
  if query.lower() == "exit" or query.lower() == "quit":
    break
  conversational_agent.run(query)


Human (type 'exit' or 'quit' to quit): who is David Guetta?

Agent custom-at-query-time started with {'query': 'who is David Guetta?', 'params': None}
<s> 
[INST] <<SYS>>
You are a helpful assistant who writes short answers.
<</SYS>>


</s><s>  [INST] who is David Guetta? [/INST]
David Guetta is a French DJ and record producer. He is known for his energetic and catchy electronic dance music (EDM) tracks, and has collaborated with numerous artists such as Sia, Rihanna, and Blackpink. He has released several successful albums and singles, and has performed at festivals and events around the world.</s>

Human (type 'exit' or 'quit' to quit): How old is he?

Agent custom-at-query-time started with {'query': 'How old is he?', 'params': None}
<s> 
[INST] <<SYS>>
You are a helpful assistant who writes short answers.
<</SYS>>


Human: who is David Guetta?
AI: 
[INST] <<SYS>>
You are a helpful assistant who writes short answers.
<</SYS>>


  [INST] who is David Guetta? [/INST]
David Guetta is 

## More (generative) ideas with Haystack

- [Building a Conversational Chat App](https://haystack.deepset.ai/tutorials/24_building_chat_app/)
- [Customizing PromptNode for NLP Tasks](https://haystack.deepset.ai/tutorials/21_customizing_promptnode)
- [Creating a Generative QA Pipeline with PromptNode](https://haystack.deepset.ai/tutorials/22_pipeline_with_promptnode)
- [Answering Multihop Questions with Agents](https://haystack.deepset.ai/tutorials/23_answering_multihop_questions_with_agents)
